<a href="https://colab.research.google.com/github/Nastarii/PCB-defect-detection/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
!pip install imageio

Upload the dataset from github

In [2]:
!wget --no-check-certificate \
    "https://github.com/tangsanli5201/DeepPCB/archive/refs/heads/master.zip" \
    -O "/tmp/DeepPCB.zip"

--2021-09-22 22:13:16--  https://github.com/tangsanli5201/DeepPCB/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tangsanli5201/DeepPCB/zip/refs/heads/master [following]
--2021-09-22 22:13:16--  https://codeload.github.com/tangsanli5201/DeepPCB/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/DeepPCB.zip’

/tmp/DeepPCB.zip        [           <=>      ]  86.79M  19.8MB/s    in 4.4s    

2021-09-22 22:13:21 (19.5 MB/s) - ‘/tmp/DeepPCB.zip’ saved [91004517]



# Libraries

---



In [3]:
import matplotlib as plt
import tensorflow as tf
import pandas as pd
import cv2 as cv
import zipfile
import imageio
import imgaug
import torch
import PIL
import os

Extract Dataset from zip

In [4]:
zip_format = zipfile.ZipFile('/tmp/DeepPCB.zip','r')
zip_format.extractall('/tmp')
zip_format.close()

In [36]:
directory = '/tmp/DeepPCB-master/PCBData/'

train_val_paths = pd.read_csv(directory + 'trainval.txt', delimiter = " ",header= None)
test_paths = pd.read_csv(directory + 'test.txt', delimiter = " ", header= None)

In [37]:
train_txt_files = train_val_paths[1][700:].values
val_txt_files = train_val_paths[1][:-300].values
test_txt_files = test_paths[1].values

In [44]:
train_imgs_path = train_val_paths[0][700:].values
val_imgs_path = train_val_paths[0][:-300].values
test_imgs_path = test_paths[0].values

array(['group20085/20085/20085292.jpg', 'group20085/20085/20085293.jpg',
       'group20085/20085/20085294.jpg', 'group20085/20085/20085295.jpg',
       'group20085/20085/20085296.jpg'], dtype=object)